In [ ]:
# default_exp core

# Calorie counter

> API details.

First you will provide your height, weight and age and calculate your Basal Metabolic Rate, the number of calories required to keep your body functioning at rest.

After choosing your usual level of activity the Active Metabolic Rate, the number of calories that we consume on a daily basis depending on our height, gender, age, weight and entered activity level whilst maintaining current weight, will be calculated.

Than you will be asked to type in the activity you were doing today. If there are more than one activities matching your input, you will select one from the lst that matches your search pattern. Additionally you will enter the time you were doing this activity.

source for the formulas:    https://www.verywellfit.com/how-many-calories-do-i-need-each-day-2506873
source for the MET values:  https://golf.procon.org/met-values-for-800-activities/

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
import pandas as pd
import regex as re


In [ ]:
#export
def setup_logger(loggerName='calorie_count_logger', logFile='log/calorie_count.log'):
    """
    Returns a logger with the specified name or "calorie_count_logger", if loggerName is None.
    Generated logs will be stored in the specified file or in log/calorie_count.log, if logFile is None


    Parameters: 

        loggerName: string
        a name for logger, that is used in getLogger

        logFile: string
        file name, where logs are stored

    Return:

        logger 
        logger used in the calorie_count functions 
 """
    import logging
    logger = logging.getLogger(loggerName)
    logger.setLevel(logging.DEBUG)
    fh = logging.FileHandler(logFile)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s', 
                        datefmt='%Y/%m/%d %I:%M:%S')
    fh.setLevel(logging.DEBUG)
    fh.setFormatter(formatter)
    logger.addHandler(fh)
    return logger
   
logger = setup_logger()


In [ ]:
#export
def input_body_parameters():
    """
    returns a dictionary with user input of body parameters: height, weight, age, gender and activity level

    Raises: 

        AssertError: if inputs of type int are negative
                
        AssertError: if inputs of type string are not 'w' or 'm'
    
    Return: 

        dict
        a dictionary with body parameters as keys and their values as values
    """
    message_activity_level = 'Please choose your level of activity:\
                \n1 - little to no exercise \
                \n2 - light exercise 1–3 days a week \
                \n3 - moderate exercise 3–5 days a week \
                \n4 - hard exercises 6–7 days a week \
                \n5 - physically demanding job or particularly challenging exercise routine'
    message_dict = {'height': 'Input height in cm',
                'weight': 'Input weight in kg',
                'age': 'Input age in full years',
                'gender': 'Input biological gender as w or m',
                'activity_level': message_activity_level} 

    input_values = [170, 56, 41,'m', 1]
    body_params = {}
    n = range(len(message_dict))
    zipped = zip(message_dict.keys(), n)
    assert len(input_values) == len(message_dict)
    for param, nr in zipped:
        if param =='gender':
            try:
                body_params[param] = input(message_dict[param])
            except:
                body_params[param] = input_values[nr]
            assert body_params[param] == 'w' or body_params[param] == 'm', 'gender value should be w or m'
        else:
            try:
                body_params[param] = int(input(message_dict[param]))
            except:
                body_params[param] = input_values[nr]
            assert body_params[param] > 0, 'Please provide a positive value'

        body_params[param] = body_params[param]

        print(f'Your {param} is {body_params[param]}')
    logger.info('user input complete')
    return body_params


def calculate_bmr_amr(body_parameters):
    """
    returns basal metabolic rate (bmr) and active metabolic rate (amr) 
    
    Parameters: 

        body_parameters: dict
        dictionary with body parameters as keys and their values as values

    Returns: 

        float
        basal metabolic rate (bmr)

        float
        active metabolic rate (amr)
    """
    dict_activity = {1:1.2, 2:1.37, 3:1.55, 4:1.725, 6:1.9}

    if body_parameters['gender'] == 'w':
        bmr = float(655.1 + (9.563 * body_parameters['weight']) +
            (1.850 * body_parameters['height']) - 
            (4.676 * body_parameters['age']))
    else:
        bmr = float(66.47 + (13.75 * body_parameters['weight']) + 
            (5.003 * body_parameters['height']) - 
            (6.755 * body_parameters['age']))

    amr = bmr * dict_activity[body_parameters['activity_level']]
    logger.info('calculate_bmr_amr complete')
    print(f'\nYour BMR is {bmr:.1f}')
    print(f'To stay at your current weight you need to consume {amr:.0f} calories')
    return bmr, amr
    

def calculate_burned_calories(body_parameters):
    """
    Returns the amount of calories burned during activity and during the day apart of activity.
    
    To calculate calories for specific activity its MET value is chosen from the table.
    To assign MET user should type in the name of her activity for the search.
    If user doesn't provide this information, activity is considered to be standing and light effort with MET of 1.4
    Calories burned during rest of the day are calculated with standard MET of 1.2.
    
    Parameters: 

        body_parameters: dict
        dictionary with body parameters as keys and their values as values

    Returns: 
    
        float
        amount of calories burned during activity

        float
        amount of calories burned during the day apart of activity
    """
    import pandas as pd
    
    met_list = pd.read_csv('data/met_list_activities.csv',
                            sep=';', 
                            #encoding='ANSI', 
                            decimal=',', 
                            error_bad_lines=False)
    nr = 0
    met = 0 
    time = 0
    your_act = ''
    while met == 0:
        try:
            your_act = input('Type in the activity you were doing today:')
        except:
            your_act = 'standing, light effort tasks'
        # cheking, if the typed activity is in the list of activities
        if len(met_list[met_list['SPECIFIC MOTION'].str.contains(your_act)]) > 1:
            print('\n', met_list[met_list['SPECIFIC MOTION'].str.contains(your_act)])
            try:
                nr = int(input('\nSelect MET of your activity from the list'))
            except:
                nr = 176
        elif len(met_list[met_list['SPECIFIC MOTION'].str.contains(your_act)]) == 0:
            print('Please type again')
        elif len(met_list[met_list['SPECIFIC MOTION'].str.contains(your_act)]) == 1:
            nr = met_list[met_list['SPECIFIC MOTION'].str.contains(your_act)].index
        met = float(met_list['METs'].iloc[nr])
        activity = str(met_list['SPECIFIC MOTION'].iloc[nr])
    try:
        time = int(input('Type in the time of your activity for today in minutes:'))
    except:
        time = 30

    burned_cal = met * body_parameters['weight'] * time/60
    bmr_cal = 1.2 * body_parameters['weight'] * (24 - time / 60)
    logger.info('calculate_burned_calories complete')
    print(f'\nDuring {time} min of {activity} you burned {burned_cal:.0f} kcal.')
    print(f'Your total daily energy expenditure was {round((burned_cal + bmr_cal), 2):.0f} kcal today')
    return burned_cal, bmr_cal

In [ ]:
#hide
# idea! add possibility to correct initial data
body_params = input_body_parameters()

bmr, amr = calculate_bmr_amr(body_params)

Your height is 190
Your weight is 87
Your age is 32
Your gender is m
Your activity is 1

Your BMR is 1997.1
To stay at your current weight you need to consume 2397 calories


In [ ]:
#hide
print('Please provide your weight, height and age in the first part of the calculator')

burned_cal, bmr_cal = calculate_burned_calories(body_params)

C:\Users\AFEDOR~1\AppData\Local\Temp/ipykernel_22884/2494668240.py:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  burned_cal, bmr_cal = calculate_burned_calories(body_params)


Please provide your weight, height and age in the first part of the calculator

                  ACTIVITY                                    SPECIFIC MOTION  \
128       home activities                    mopping, standing, light effort   
157       home activities  laundry, fold or hang clothes, put clothes in ...   
174       home activities  standing, light effort tasks (pump gas, change...   
439            occupation  standing, light/moderate effort (e.g., assembl...   
805  volunteer activities  standing, light work (filing, talking, assembl...   
810  volunteer activities  standing, light/moderate work (e.g., pack boxe...   

     METs  
128   2.5  
157   2.0  
174   2.0  
439   3.0  
805   2.3  
810   3.0  

During 24 min of standing, light effort tasks (pump gas, change light bulb, etc.) you burned 18 kcal.
Your total daily energy expenditure was 670 kcal today


### Notion API and visualisation

In [ ]:
# export
class NotionSync:
    def __init__(self, db_token, db_id):
        """
        Class for Notion API

        Parameters:
        db_token 
        security token from Notion API

        db_id
        database id from Notion API
        """
        self._db_id = db_id
        self._db_token = db_token
        self._headers = {
                "Authorization": "Bearer " + self._db_token,
                "Content-Type": "application/json",
                "Notion-Version": "2022-02-22",
                "Accept": "application/json"
            }

    def query_databases(self):
        """
        Returns json structured data with the content from NotionSync object.

        Returns: 
            json
            data from NotionSync object
    
        """
        import requests
        url = f"https://api.notion.com/v1/databases/{self._db_id}/query"
        response = requests.post(url, headers=self._headers)
        if response.status_code != 200:
            raise ValueError(f'Response Status: {response.status_code}')
        else:
            return response.json()
        

    def get_projects_titles(self, data_json):
        """
        Returns a list of property titles from a json data object.

        Parameters:

            data_json
            data from NotionSync object

        Returns:
            list
            list of property titles        
        """
        return list(data_json["results"][0]["properties"].keys())

    def add_entry(self, entry):
        """
        Writes a new entry to the data base.

        Parameters:
            entry
            a dictionary with property titles as keys and user inputs as values
        """
        import requests, json
        updateUrl = "https://api.notion.com/v1/pages/"
        updateData = {
            'parent': {'database_id': 'e3aef208-01fe-4934-968a-5cf86742658f'},
            'properties': {
                'height': {
                    "rich_text": [
                        {
                            "text": {
                                "content": entry['height']
                            }
                        }
                    ]      
                },
                'weight': {
                    "rich_text": [
                        {
                            "text": {
                                "content": entry['weight']
                            }
                        }
                    ]      
                },
                'age': {
                    "rich_text": [
                        {
                            "text": {
                                "content": entry['age']
                            }
                        }
                    ]      
                },
                'gender': {
                    "rich_text": [
                        {
                            "text": {
                                "content": entry['gender']
                            }
                        }
                    ]      
                },
                'activity_level': {
                    "rich_text": [
                        {
                            "text": {
                                "content": entry['activity_level']
                            }
                        }
                    ]      
                },
                'activity': {
                    "rich_text": [
                        {
                            "text": {
                                "content": entry['activity']
                            }
                        }
                    ]      
                },
                'time': {
                    "rich_text": [
                        {
                            "text": {
                                "content": entry['time']
                            }
                        }
                    ]      
                }
            }
        }

        data = json.dumps(updateData)
        response = requests.post(updateUrl, headers=self._headers, data=data)
        if response.status_code != 200:
            raise ValueError(f'Response Status: {response.status_code}\nResponse Text: {response.text}')
        else:
            print(response.status_code)

In [ ]:
# export
def extract_data_to_df(data_json):
    """
    Returns a DataFrame with all activities from data base and respective time.

    Parameters:
        data_json
        data from the data base

    Returns:
        DataFrame
        a DataFrame with columns activity and time with a row for each entry in the data base
    """
    from pandas import DataFrame
    activity = []
    time = []
    for i in range(len(data_json['results'])):
        activity.append((
            data_json['results'][i]['properties']['activity']['rich_text'][0]['text']['content'],
            int(data_json['results'][i]['properties']['time']['rich_text'][0]['text']['content'])))
    df_activity = DataFrame(activity,  columns=['activity', 'time'])
    return(df_activity)


def get_activity_sum(df):
    """
    Returns a DataFrame with summed time for each activity and number of user, who did it, in the data base.

    Parameters:
        df
        DataFrame with activity information, must contain columns 'activity', 'time'

    Result:
        df_sum
        DataFrame with columns activity, time, number_user 

    """
    import pandas as pd
    
    df_sum = df.groupby('activity').sum()
    c = df.groupby('activity').count()
    df_sum = pd.concat([df_sum, c], axis=1)
    df_sum.columns = ['time', 'number_user']
    df_sum = df_sum.reset_index()
    return df_sum

def show_hist(df_sum):
    """
    Returns a barplot with all activities from the data base and overall time for each of these activities

    Parameters:
        df_sum
        DataFrame with activities and respective summed times

    Returns:
        plotly.graph_objs._figure.Figure
        a barplot plotly figure
    """
    import plotly.express as px
    labels = df_sum['labels'] = [(i[:30] + '..') if len(i) > 30 else i for i in df_sum['activity']]
    fig = px.bar(df_sum,
                x="labels",
                y="time", 
                labels=dict(labels="Acivity", time='Length of activity, min', color=''),
                color='activity',
                title='Total activity on the website in min',
                text_auto=True)
    fig.update_layout(showlegend=False)
    return fig